In [1]:
# Q1

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("homework_5") \
    .getOrCreate()

print(spark.version)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 02:25:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.5.4


In [3]:
# Q2

In [4]:
df_yellow = spark.read.parquet('yellow_tripdata_2024-10.parquet')

df_yellow_repartition = df_yellow.repartition(4)

df_yellow_repartition.write.parquet('yellow_tripdata_2024-10_repartition.parquet', mode='overwrite')

In [5]:
# ls -lh yellow_tripdata_2024-10_repartition.parquet
# total 188928
# -rw-r--r--  1 jingwenxu  staff     0B Mar  6 02:02 _SUCCESS
# -rw-r--r--  1 jingwenxu  staff    22M Mar  6 02:02 part-00000-a27c4ca0-b528-46dd-b754-906f5bec9a16-c000.snappy.parquet
# -rw-r--r--  1 jingwenxu  staff    22M Mar  6 02:02 part-00001-a27c4ca0-b528-46dd-b754-906f5bec9a16-c000.snappy.parquet
# -rw-r--r--  1 jingwenxu  staff    22M Mar  6 02:02 part-00002-a27c4ca0-b528-46dd-b754-906f5bec9a16-c000.snappy.parquet
# -rw-r--r--  1 jingwenxu  staff    22M Mar  6 02:02 part-00003-a27c4ca0-b528-46dd-b754-906f5bec9a16-c000.snappy.parquet

In [6]:
# 3

In [7]:
df_yellow = spark.read.parquet('yellow_tripdata_2024-10.parquet')
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [20]:
df_yellow.registerTempTable('yellow')

num_taxi = spark.sql("""
SELECT 
    COUNT(*) AS num_tax
FROM 
    yellow
WHERE 
    DATE(tpep_pickup_datetime) = '2024-10-15'
""")

num_taxi.show()

+-------+
|num_tax|
+-------+
| 128893|
+-------+



In [10]:
# 4

In [11]:
df_yellow.createOrReplaceTempView('yellow')

spark.sql("""
SELECT 
    MAX((unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600) AS max_trip_hours
FROM 
    yellow
""").show()


+------------------+
|    max_trip_hours|
+------------------+
|162.61777777777777|
+------------------+



In [12]:
# 5 -> 4040

In [13]:
# 6

In [17]:
df_zone = spark.read.csv('taxi_zone_lookup.csv', header=True, inferSchema=True)
df_zone.registerTempTable('zones')
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [16]:
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [18]:
spark.sql("""
SELECT 
    y.PULocationID, z.Zone, COUNT(*)
FROM
    yellow y
JOIN
    zones z
ON
    y.PULocationID = z.LocationID
GROUP BY
    1, 2
ORDER BY 3
LIMIT 1
""").show()

+------------+--------------------+--------+
|PULocationID|                Zone|count(1)|
+------------+--------------------+--------+
|         105|Governor's Island...|       1|
+------------+--------------------+--------+

